In [18]:
import re
import os
import urllib
from datetime import datetime
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


In [19]:
regions = ['Cherkasy', 'Chernihiv', 'Chernivtsi', 'Crimea', "Dnipropetrovs'k", "Donets'k", "Ivano-Frankivs'k", 'Kharkiv', 'Kherson', "Khmel'nyts'kyy", 'Kiev', 'Kie', 'Kirovohrad', "Luhans'k", "L'viv", 'Mykolayiv', 'Odessa', 'Poltava', 'Rivne', "Sevastopol'", 'Sumy', "Ternopil'", 'Transcarpathia', 'Vinnytsya', 'Volyn', 'Zaporizhzhya', 'Zhytomyr',]

In [20]:
def get_data_by_id(i):
    url=f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2022&type=Mean'
    wp = urllib.request.urlopen(url)
    text = wp.read()
    with open(f'star/UKR_{i}_{datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}.csv', 'wb') as file:
        file.write(text)

In [23]:
for i in range(1,28):
    get_data_by_id(i)

In [35]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


def create_dataframe(path):
    concatenate_files=[]
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f): 
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'area']
            df = pd.read_csv(f, header = 1, names = headers)
            
            df['Year']=df['Year'].apply(lambda cw : remove_tags(cw))
            df = df.drop(df.loc[df['VHI'] == -1].index)
            df = df[df['Week'].notna()]
            df['area'] = f.split('_')[1]
            
            concatenate_files.append(df)
    return pd.concat(concatenate_files, axis=0, ignore_index=True)

In [36]:
df = create_dataframe('./star')
df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
55021,2022,4.0,0.099,266.96,57.74,32.16,44.95,15
55022,2022,5.0,0.106,267.92,56.78,33.91,45.34,15
55023,2022,6.0,0.116,269.46,56.76,33.72,45.24,15
55024,2022,7.0,0.132,271.47,60.39,30.71,45.55,15
55025,2022,8.0,0.147,272.78,63.43,31.12,47.27,15


In [25]:
def change_id_to_name(i):
    return regions[int(i)-1]
df["area"] = df["area"].apply(change_id_to_name)
df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
55021,2022,4.0,0.099,266.96,57.74,32.16,44.95,L'viv
55022,2022,5.0,0.106,267.92,56.78,33.91,45.34,L'viv
55023,2022,6.0,0.116,269.46,56.76,33.72,45.24,L'viv
55024,2022,7.0,0.132,271.47,60.39,30.71,45.55,L'viv
55025,2022,8.0,0.147,272.78,63.43,31.12,47.27,L'viv


In [37]:
def change_id_to_name_2():
    for i in set(df.area):
        df["area"].replace({i:regions[int(i)-1]}, inplace = True)
change_id_to_name_2()
df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
55021,2022,4.0,0.099,266.96,57.74,32.16,44.95,L'viv
55022,2022,5.0,0.106,267.92,56.78,33.91,45.34,L'viv
55023,2022,6.0,0.116,269.46,56.76,33.72,45.24,L'viv
55024,2022,7.0,0.132,271.47,60.39,30.71,45.55,L'viv
55025,2022,8.0,0.147,272.78,63.43,31.12,47.27,L'viv


In [81]:
index="Zaporizhzhya"
year='2022'
VHI = df[(df["area"] == index) & (df["Year"] == year)]['VHI']
print(VHI, '\n',  VHI.max(), VHI.min())

6106    44.22
6107    44.16
6108    43.91
6109    41.53
6110    40.69
6111    39.29
6112    38.23
6113    37.95
Name: VHI, dtype: float64 
 44.22 37.95


In [82]:
set(df[((df["area"] == index) & (df.VHI <= 15))]['Year'])

{'2007'}